# Pipeline For Random Forest Classifier

In [ ]:
from pyspark import SQLContext, SparkConf, SparkContext
import pyspark.sql.functions as F
from pypspark.ml.feature import VectorAssembler, Vector Indexer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classfication import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassficationEvaluator

In [ ]:
#reading data
#change this to suit your data
df = sqlCtx.read... 

In [ ]:
df.printSchema()

In [ ]:
#if your data has columns that contain strings then 
#an index value needs to assigned to each unique value 
#so as to input it to a classfier

In [ ]:
#as my data had multiple string columns therefore I developed a list of string indexers
string_indexer_list = [StringIndexer(inputCol=i[0], outputCol=i[0] + '_index') for i in df.dtypes if i[1]=='string' 
                       and i[0] not in [#specify columns not to be included as inputs to models like customer_name]]
#when we will fit this to the data then it will create new columns with indices

In [ ]:
#input columns you want in the feature vector
#note: include the columns that contain indices instead of original string columns
cols = [#example:'gender_index', 'total_after_tax']

In [ ]:
#now we will assemble the features into a sparse feature vector as the classfier only takes in this format
assembler = VectorAssembler(inputCols=cols, outputCol='features')

In [ ]:
#vector indexer adds indices to features with unique values below a certain threshhold, essentially makes them into categorical variables
vector_indexer = VectorIndexer(inputCols='features', 
                               outputCol='indexed_features', 
                               maxCategories=10)

In [ ]:
#adding indices to the label column
label_indexer = StringIndexer(inputCol='insert the name of the target variable', 
                              outputCol='label')

In [ ]:
#classifier object
rfc = RandomForestClassfier(featuresCol='indexed_features',
                            labelCol='label',
                            numTrees=100)

In [ ]:
#putting all the objects into the pipeline
pipeline = Pipeline[stages= [*string_indexer_list, assembler, 
                             vector_indexer,label_indexer, rfc]]

In [ ]:
#train test split
train, test = df.randomSplit([.8,.2])

In [ ]:
#fitting the model
model = pipeline.fit(train)

In [ ]:
#prediction
prediction = model.transform(test)

In [ ]:
#calculating accuracy
accuracy = MulticlassClassificationEvaluator(labelCol='label', predictioCol='prediction'
                                            metricName='accuracy')
accuracy.evaluate(prediction)

In [ ]:
#confusion metric heatmap
_heatmap_matrix = prediction.groupBy('prediction', 'label').count().toPandas()
heatmap_matrix = _heatmap_matrix.pivot(index ='label', columns='prediction', values='count')
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
sns.heatmap(heatmap_matrix/heatmap_matrix.sum(), cmap=cmap)